# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
pip install geoviews

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,erdenet,49.0333,104.0833,-0.63,94,20,1.17,MN,1696279473
1,1,port alfred,-33.5906,26.8910,16.77,91,0,0.45,ZA,1696279473
2,2,margate,51.3813,1.3862,19.06,94,100,0.89,GB,1696279473
3,3,avarua,-21.2078,-159.7750,22.03,73,75,4.12,CK,1696279474
4,4,pangody,65.8500,74.4859,9.12,89,100,8.37,RU,1696279474


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
import pandas as pd
import geoviews as gv
import geoviews.tile_sources as gts

# Import API key
from api_keys import geoapify_key

# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Define the Geoapify API endpoint for geocoding
geoapify_endpoint = f"https://api.geoapify.com/v2/geocoding/search?apiKey={geoapify_key}"

# Create a Points Element from the city_data_df DataFrame with color and size based on humidity
points = gv.Points(city_data_df, ['Lng', 'Lat'], ['City', 'Humidity', 'Country']).opts(
    size=gv.dim('Humidity') * 0.1,  # Adjust the size scaling factor as needed
    color='Humidity',
    cmap='viridis',
    colorbar=True,
    width=800,
    height=500,
    tools=['hover'],
    title='City Humidity'
)

# Create a dynamic map
map_plot = (gts.CartoDark * points).redim.range(
    Lat=(city_data_df['Lat'].min(), city_data_df['Lat'].max()),
    Lng=(city_data_df['Lng'].min(), city_data_df['Lng'].max())
)

# Display the map
map_plot


C:\Users\mklut\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
city_data_df.head(6)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,erdenet,49.0333,104.0833,-0.63,94,20,1.17,MN,1696279473
1,1,port alfred,-33.5906,26.8910,16.77,91,0,0.45,ZA,1696279473
2,2,margate,51.3813,1.3862,19.06,94,100,0.89,GB,1696279473
3,3,avarua,-21.2078,-159.7750,22.03,73,75,4.12,CK,1696279474
4,4,pangody,65.8500,74.4859,9.12,89,100,8.37,RU,1696279474
5,5,waitangi,-43.9535,-176.5597,11.57,88,95,0.89,NZ,1696279474


In [6]:
# Display summary statistics for columns of interest
summary_stats = city_data_df[['Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed']].describe()
print(summary_stats)


         Max Temp    Humidity  Cloudiness  Wind Speed
count  572.000000  572.000000  572.000000  572.000000
mean    19.489703   70.286713   54.821678    3.721888
std      9.439292   20.553845   40.223452    2.538013
min     -8.360000    8.000000    0.000000    0.000000
25%     12.957500   60.750000    9.750000    1.790000
50%     21.220000   76.000000   66.000000    3.090000
75%     26.650000   86.000000   99.000000    5.075000
max     40.200000  100.000000  100.000000   15.240000


In [14]:
#Filter destinations should be between 20°C to 25°C, Humidity below 85%, Cloudiness equal to or below 10%, and Wind Speed below 5 m/s
filtered_cities_df = city_data_df[(city_data_df['Max Temp'] >= 20) & (city_data_df['Max Temp'] <= 25) &
                                   (city_data_df['Humidity'] < 85) & (city_data_df['Cloudiness'] <= 10) &
                                   (city_data_df['Wind Speed'] < 5)]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df.head(10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,15,santa maria,-29.6842,-53.8069,20.61,49,0,4.63,BR,1696279475
42,42,saint-francois,46.4154,3.9054,20.69,60,0,3.09,FR,1696279478
61,61,colonia,50.9333,6.9500,20.53,71,0,2.57,DE,1696279480
94,94,saint-pierre,-21.3393,55.4781,21.82,78,0,2.57,RE,1696279484
145,145,pisco,-13.7000,-76.2167,23.03,68,0,4.12,PE,1696279489
248,248,ensenada,31.8667,-116.6167,21.64,69,0,4.68,MX,1696278929
268,268,sydney,-33.8679,151.2073,21.84,82,0,3.09,AU,1696279329
269,269,rizokarpaso,35.6000,34.3750,24.57,62,1,4.39,CY,1696279504
271,271,kolonia,50.9333,6.9500,20.53,71,0,2.57,DE,1696279480
277,277,mpika,-11.8343,31.4529,21.25,43,10,4.24,ZM,1696279505


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head(10)


,City,Country,Lat,Lng,Humidity,Hotel Name
15,santa maria,BR,-29.6842,-53.8069,49,
42,saint-francois,FR,46.4154,3.9054,60,
61,colonia,DE,50.9333,6.9500,71,
94,saint-pierre,RE,-21.3393,55.4781,78,
145,pisco,PE,-13.7000,-76.2167,68,
248,ensenada,MX,31.8667,-116.6167,69,
268,sydney,AU,-33.8679,151.2073,82,
269,rizokarpaso,CY,35.6000,34.3750,62,
271,kolonia,DE,50.9333,6.9500,71,
277,mpika,ZM,-11.8343,31.4529,43,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Import necessary libraries
import pandas as pd
import requests
from api_keys import geoapify_key

# Define the Geoapify API endpoint for hotel search
hotel_search_endpoint = "https://api.geoapify.com/v2/places"


# Define the parameters for the API request
params = {
    'categories': 'accommodation.hotel',
    'filter': f'circle:{hotel_df["Lng"].iloc[0]},{hotel_df["Lat"].iloc[0]},10000',  # Using the first city's coordinates
    'bias': f'proximity:{hotel_df["Lng"].iloc[0]},{hotel_df["Lat"].iloc[0]}',
    'limit': 20,  # Adjust the limit as needed
    'apiKey': geoapify_key
}

# Add a new column 'Hotel Name' to hotel_df
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

# Iterate through each row in the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Update the filter and bias parameters with the city's coordinates
    params['filter'] = f'circle:{row["Lng"]},{row["Lat"]},10000'
    params['bias'] = f'proximity:{row["Lng"]},{row["Lat"]}'

    # Make the API request
    response = requests.get(hotel_search_endpoint, params=params)
    data = response.json()

    # Check if 'features' key exists and if the first feature has a 'name' key
    if 'features' in data and data['features'] and 'name' in data['features'][0]['properties']:
        hotel_name = data['features'][0]['properties']['name']
    else:
        hotel_name = "No hotel found"

    # Update the "Hotel Name" column in the DataFrame with the hotel_name
    hotel_df.at[index, 'Hotel Name'] = hotel_name

    # Display the result
    print(f"{row['City']} - nearest hotel: {hotel_name}")


santa maria - nearest hotel: Hotel Morotin
saint-francois - nearest hotel: Chez Lily
colonia - nearest hotel: Wasserturm Hotel Cologne
saint-pierre - nearest hotel: Tropic Hotel
pisco - nearest hotel: La Portada
ensenada - nearest hotel: Hotel San Diego
sydney - nearest hotel: The Westin
rizokarpaso - nearest hotel: Villa Carparis
kolonia - nearest hotel: Wasserturm Hotel Cologne
mpika - nearest hotel: Melody Lodge
roccella ionica - nearest hotel: Mediterraneo
rio grande - nearest hotel: Hotel Vila Moura Executivo
fortuna - nearest hotel: Comfort Inn & Suites Redwood Country
willowmore - nearest hotel: No hotel found
pacific grove - nearest hotel: Pacific Grove Inn
monte patria - nearest hotel: Hotel Monte Cordillera
sulmona - nearest hotel: Hotel Albergo Stella Ristorante
tamandare - nearest hotel: Pousada Baia dos Corais
ceerigaabo - nearest hotel: Sanaag Hotel
pierre - nearest hotel: Quality Inn
mahdia - nearest hotel: La Maison Bleue
middelkerke - nearest hotel: Hotel Middelpunt


In [10]:
# Display the updated DataFrame
hotel_df.head(5)

,City,Country,Lat,Lng,Humidity,Hotel Name
15,santa maria,BR,-29.6842,-53.8069,49,Hotel Morotin
42,saint-francois,FR,46.4154,3.9054,60,Chez Lily
61,colonia,DE,50.9333,6.9500,71,Wasserturm Hotel Cologne
94,saint-pierre,RE,-21.3393,55.4781,78,Tropic Hotel
145,pisco,PE,-13.7000,-76.2167,68,La Portada


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
# Save hotel_df to a CSV file
hotel_df.to_csv('hotel_data.csv', index=False)
hotel_df = pd.read_csv("hotel_data.csv")

In [12]:
import pandas as pd
import geoviews as gv
import geoviews.tile_sources as gts

# Import API key
from api_keys import geoapify_key

# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Define the Geoapify API endpoint for geocoding
geoapify_endpoint = f"https://api.geoapify.com/v2/geocoding/search?apiKey={geoapify_key}"

# Create a Points Element from the city_data_df DataFrame with color and size based on humidity
points = gv.Points(hotel_df, ['Lng', 'Lat'], ['City', 'Humidity', 'Country','Hotel Name']).opts(
    size=gv.dim('Humidity') * 0.1,  # Adjust the size scaling factor as needed
    color='Humidity',
    cmap='viridis',
    colorbar=True,
    width=800,
    height=500,
    tools=['hover'],
    title='City Humidity'
)

# Create a dynamic map
map_plot = (gts.CartoDark * points).redim.range(
    Lat=(city_data_df['Lat'].min(), city_data_df['Lat'].max()),
    Lng=(city_data_df['Lng'].min(), city_data_df['Lng'].max())
)

# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)